## Inputs

1. restaurant dimensions
1. number of timeslots?
1. marginal revenue function
1. cost of goodwill
1. randomly generated data - boolean
1. maximum table size
1. objective? - enum

## Construction Heuristic

### Algorithm

1. sort parties in order of descending size
1. add parties to the timeslot k until the restaurant capacity is full or no more tables can be added

### Step 1: Parse input parameters

In [1]:
import json
import random

w = l = customer_goodwill = REVENUE_GOAL = num_timeslots = randomly_generated_data = 0
marginal_revenue = {}

# set of timeslots - static
K = 7

with open('../data.json') as f:
    data = json.load(f)
    w = data['restaurant_width']
    l = data['restaurant_length']
    customer_goodwill = data['customer_goodwill']
    REVENUE_GOAL = data['revenue_goal']
    randomly_generated_data = data['randomly_generated_data']
    max_party_size = data['max_party_size']
    for i in data['marginal_revenue']:
        marginal_revenue[int(i)] = data['marginal_revenue'][i]
        
# Restaurant squared area
MAX_SIZE = l * w

### Step 2: Randomly generate data

In [2]:
# set of parties at each timeslot
# since there are 7 timeslots and since this data is randomly generated, 
# the set of parties during timelsot k will be I[k].
# Prior to having randomly generated demand we had set I = 40 (40 parties) for each timeslot
if randomly_generated_data == True:
    I = [
        random.randint(10,15), 
        random.randint(17,24),
        random.randint(17,24),
        random.randint(10,15),
        random.randint(25,40),
        random.randint(25,40),
        random.randint(10,15),
    ]

else:
    print("Please allow data to be randomly generated to ensure accuracy and reliability of results")


#initializing the party 
party_size = {}
for k in range(K):
    for i in range(I[k]):
        party_size[i, k] = random.randint(2, max_party_size)
        
print(f"{max_party_size}\n")
print(f"{party_size}")

8

{(0, 0): 3, (1, 0): 4, (2, 0): 5, (3, 0): 3, (4, 0): 5, (5, 0): 2, (6, 0): 7, (7, 0): 4, (8, 0): 7, (9, 0): 2, (10, 0): 4, (0, 1): 5, (1, 1): 4, (2, 1): 6, (3, 1): 2, (4, 1): 3, (5, 1): 3, (6, 1): 2, (7, 1): 4, (8, 1): 3, (9, 1): 6, (10, 1): 2, (11, 1): 7, (12, 1): 4, (13, 1): 3, (14, 1): 5, (15, 1): 6, (16, 1): 2, (17, 1): 6, (18, 1): 4, (19, 1): 2, (20, 1): 3, (21, 1): 3, (22, 1): 6, (23, 1): 6, (0, 2): 6, (1, 2): 5, (2, 2): 2, (3, 2): 7, (4, 2): 8, (5, 2): 6, (6, 2): 8, (7, 2): 5, (8, 2): 7, (9, 2): 6, (10, 2): 3, (11, 2): 4, (12, 2): 3, (13, 2): 4, (14, 2): 4, (15, 2): 4, (16, 2): 8, (17, 2): 5, (18, 2): 4, (19, 2): 3, (20, 2): 5, (21, 2): 5, (22, 2): 8, (0, 3): 3, (1, 3): 2, (2, 3): 5, (3, 3): 2, (4, 3): 5, (5, 3): 4, (6, 3): 7, (7, 3): 2, (8, 3): 5, (9, 3): 4, (10, 3): 4, (11, 3): 2, (0, 4): 6, (1, 4): 5, (2, 4): 8, (3, 4): 2, (4, 4): 7, (5, 4): 5, (6, 4): 5, (7, 4): 7, (8, 4): 2, (9, 4): 8, (10, 4): 7, (11, 4): 5, (12, 4): 6, (13, 4): 8, (14, 4): 2, (15, 4): 3, (16, 4): 7, (1

### Step 3: Calculate table sizes and space that each party will occupy if seated

In [3]:
# the table mapping below represents
# how many tables a specific party_size would require
table_mapping = {}
for i in range(2,max_party_size + 1):
    table_mapping[i] = i // 2 + i % 2

# initializing the amount of space that each of the party sizes 
# would take up
space = {}
for k in range(K):
    for i in range(I[k]):
        num_tables = table_mapping.get(party_size[i, k])
        space[i, k] = 6 + 3 * num_tables

### Step 4: Create and pre-populate data structures

In [4]:
sorted_space = space.copy()

sorted_space = {k: v for k, v in sorted(sorted_space.items(), key=lambda item: item[1], reverse=True)}
sorted_space = {k: v for k, v in sorted(sorted_space.items(), key=lambda item: item[0][1], reverse=False)}
print(f"{sorted_space}")

{(6, 0): 18, (8, 0): 18, (2, 0): 15, (4, 0): 15, (0, 0): 12, (1, 0): 12, (3, 0): 12, (7, 0): 12, (10, 0): 12, (5, 0): 9, (9, 0): 9, (11, 1): 18, (0, 1): 15, (2, 1): 15, (9, 1): 15, (14, 1): 15, (15, 1): 15, (17, 1): 15, (22, 1): 15, (23, 1): 15, (1, 1): 12, (4, 1): 12, (5, 1): 12, (7, 1): 12, (8, 1): 12, (12, 1): 12, (13, 1): 12, (18, 1): 12, (20, 1): 12, (21, 1): 12, (3, 1): 9, (6, 1): 9, (10, 1): 9, (16, 1): 9, (19, 1): 9, (3, 2): 18, (4, 2): 18, (6, 2): 18, (8, 2): 18, (16, 2): 18, (22, 2): 18, (0, 2): 15, (1, 2): 15, (5, 2): 15, (7, 2): 15, (9, 2): 15, (17, 2): 15, (20, 2): 15, (21, 2): 15, (10, 2): 12, (11, 2): 12, (12, 2): 12, (13, 2): 12, (14, 2): 12, (15, 2): 12, (18, 2): 12, (19, 2): 12, (2, 2): 9, (6, 3): 18, (2, 3): 15, (4, 3): 15, (8, 3): 15, (0, 3): 12, (5, 3): 12, (9, 3): 12, (10, 3): 12, (1, 3): 9, (3, 3): 9, (7, 3): 9, (11, 3): 9, (2, 4): 18, (4, 4): 18, (7, 4): 18, (9, 4): 18, (10, 4): 18, (13, 4): 18, (16, 4): 18, (18, 4): 18, (19, 4): 18, (34, 4): 18, (35, 4): 18, (3

In [5]:
# initializing a list of keys for the space dictionary
keys_list = list(sorted_space)


# looping over each of the time slot's sorted party sizes 
timeslot_sorted_parties = {}
global_counter = 0
for k in range(K):
    counter = 0
    sorted_parties=[]
    for key in sorted_space.keys():
        sorted_parties.append(sorted_space[keys_list[global_counter]])
        counter += 1
        global_counter += 1
        if(counter == I[k]):
            break
    timeslot_sorted_parties[k] = sorted_parties
print(f"{timeslot_sorted_parties}")
#print(f"{sorted_space}")
#print(f"{sorted_space[sorted_space[i],i]}")

x = [[0 for i in range(I[k])] for k in range(K)]

{0: [18, 18, 15, 15, 12, 12, 12, 12, 12, 9, 9], 1: [18, 15, 15, 15, 15, 15, 15, 15, 15, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 9, 9, 9, 9, 9], 2: [18, 18, 18, 18, 18, 18, 15, 15, 15, 15, 15, 15, 15, 15, 12, 12, 12, 12, 12, 12, 12, 12, 9], 3: [18, 15, 15, 15, 12, 12, 12, 12, 9, 9, 9, 9], 4: [18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 12, 12, 12, 12, 12, 12, 12, 12, 12, 9, 9, 9, 9, 9], 5: [18, 18, 18, 18, 18, 18, 18, 18, 18, 15, 15, 15, 15, 15, 15, 15, 15, 12, 12, 12, 12, 12, 12, 12, 12, 12, 9, 9, 9, 9, 9], 6: [18, 18, 18, 18, 18, 15, 12, 9, 9, 9]}


### Step 5: Construction Heuristic

In [42]:
# Observation: Sometimes we have a party size of 3/5 instead of 4/6 get allocated even though 4 will produce more revenue.
# This is because the space occupied by each is the same.
# Way we can do this is by sorting at party_size instead of space and that will propogate.
binaries = []
daily_revenue = 0
counter = 0
for k in range(K):
    revenue = 0
    occupied_space = 0
    print(f"timeslot{k}")
    for i in range(I[k]):
        #dict_to_append = {}
        #dict_to_append[i,k] = [0, 0]
        size_of_party =party_size[keys_list[counter]]
        if (occupied_space + timeslot_sorted_parties[k][i] < MAX_SIZE):
            occupied_space += timeslot_sorted_parties[k][i]
            x[k][i] = 1
            added_revenue = marginal_revenue[size_of_party] * size_of_party
            revenue += added_revenue
            #dict_to_append[i,k] = [1, added_revenue]
            binaries.append([i,size_of_party,k,1,added_revenue,revenue])

            print(f"party size = {size_of_party}, marginal revenue = {marginal_revenue[size_of_party]}, rev={revenue}")
        else:
            binaries.append([i,size_of_party,k,'',0,revenue])
        counter+=1
    print(f"{occupied_space}\n")
    

timeslot0
party size = 7, marginal revenue = 29, rev=203
party size = 7, marginal revenue = 29, rev=406
party size = 5, marginal revenue = 24, rev=526
party size = 5, marginal revenue = 24, rev=646
party size = 3, marginal revenue = 23, rev=715
party size = 4, marginal revenue = 22, rev=803
party size = 3, marginal revenue = 23, rev=872
party size = 4, marginal revenue = 22, rev=960
party size = 4, marginal revenue = 22, rev=1048
party size = 2, marginal revenue = 26, rev=1100
party size = 2, marginal revenue = 26, rev=1152
144

timeslot1
party size = 7, marginal revenue = 29, rev=203
party size = 5, marginal revenue = 24, rev=323
party size = 6, marginal revenue = 28, rev=491
party size = 6, marginal revenue = 28, rev=659
party size = 5, marginal revenue = 24, rev=779
party size = 6, marginal revenue = 28, rev=947
party size = 6, marginal revenue = 28, rev=1115
party size = 6, marginal revenue = 28, rev=1283
party size = 6, marginal revenue = 28, rev=1451
party size = 4, marginal reve

### Step 6: Simulated Annealing

In [43]:
for i in binaries:
    print(f"{i}")

#for key in sorted_space.keys():
    #print(f"{key}")
 #   print(f"{x[key[0]][key[1]]}")

[0, 7, 0, 1, 203, 203]
[1, 7, 0, 1, 203, 406]
[2, 5, 0, 1, 120, 526]
[3, 5, 0, 1, 120, 646]
[4, 3, 0, 1, 69, 715]
[5, 4, 0, 1, 88, 803]
[6, 3, 0, 1, 69, 872]
[7, 4, 0, 1, 88, 960]
[8, 4, 0, 1, 88, 1048]
[9, 2, 0, 1, 52, 1100]
[10, 2, 0, 1, 52, 1152]
[0, 7, 1, 1, 203, 203]
[1, 5, 1, 1, 120, 323]
[2, 6, 1, 1, 168, 491]
[3, 6, 1, 1, 168, 659]
[4, 5, 1, 1, 120, 779]
[5, 6, 1, 1, 168, 947]
[6, 6, 1, 1, 168, 1115]
[7, 6, 1, 1, 168, 1283]
[8, 6, 1, 1, 168, 1451]
[9, 4, 1, 1, 88, 1539]
[10, 3, 1, 1, 69, 1608]
[11, 3, 1, 1, 69, 1677]
[12, 4, 1, 1, 88, 1765]
[13, 3, 1, 1, 69, 1834]
[14, 4, 1, 1, 88, 1922]
[15, 3, 1, 1, 69, 1991]
[16, 4, 1, '', 0, 1991]
[17, 3, 1, '', 0, 1991]
[18, 3, 1, '', 0, 1991]
[19, 2, 1, '', 0, 1991]
[20, 2, 1, '', 0, 1991]
[21, 2, 1, '', 0, 1991]
[22, 2, 1, '', 0, 1991]
[23, 2, 1, '', 0, 1991]
[0, 7, 2, 1, 203, 203]
[1, 8, 2, 1, 240, 443]
[2, 8, 2, 1, 240, 683]
[3, 7, 2, 1, 203, 886]
[4, 8, 2, 1, 240, 1126]
[5, 8, 2, 1, 240, 1366]
[6, 6, 2, 1, 168, 1534]
[7, 5, 2, 1, 120,

In [44]:
import csv

with open('dict.csv', 'w') as csv_file:  
    writer = csv.writer(csv_file)
    writer.writerow(["Party i", "Size""Timeslot k", "seated (binary)", "added revenue"])
    for i in binaries:
       writer.writerow(i)